In [ ]:
!pip install pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TextPreprocessing") \
    .getOrCreate()

spark


In [3]:
teencode_dict = {
        'j': 'gì', 'k': 'không', 'm': 'mình', 'qhe': 'quan hệ', 'ny': 'người yêu',
        'vl': 'vãi lắm', 'khum': 'không', 'ck': 'chồng', 'vk': 'vợ', 'phimsex': 'phim người lớn',
        'tqtq': 'thích quá trời quá đất', 'bt': 'bình thường', 'qlq': 'quá là quạo', 'dc': 'được',
        'mn': 'mọi người', 'mk': 'mật khẩu', 'add': 'thêm', 'rep': 'trả lời', 'ib': 'nhắn tin riêng',
        'bb': 'bạn bè', 'stt': 'status', 'cmt': 'bình luận', 'snsd': 'sao nói sai được', 'chz': 'chuyện',
        'bvs': 'băng vệ sinh', 'chs': 'chuyện', 'ckx': 'chắc', 'cvc': 'công việc', 'tk': 'tài khoản',
        'tks': 'cảm ơn', 'y/c': 'yêu cầu', 'hpbd': 'happy birthday', 'oh': 'ồ', 'ka': 'không ai',
        'cái nè': 'cái này', 'cbi': 'chuẩn bị', 'bn': 'bạn', 'vlq': 'vãi luôn quá', 'z': 'vậy',
        'hqua': 'hôm qua', 'hnay': 'hôm nay', 'hn': 'hôm nay', 'vch': 'vãi chưởng', 'cmnr': 'chuẩn cơm mẹ nấu',
        'thks': 'cảm ơn', 'plz': 'làm ơn', 'ms': 'mới', 'r': 'rồi', 'lmk': 'làm ơn biết',
        'dcmm': 'được chưa mày', 'okie': 'ổn', 'cn': 'cuối tuần', 'dc ch': 'được chứ', 'sml': 'sấp mặt luôn',
        'pp': 'tạm biệt', 'haha': 'cười lớn', 'lol': 'cười lớn', 'gt': 'giới thiệu', 'nx': 'nhận xét',
        'bl': 'bình luận', 'aiya': 'ồ', 'zui': 'vui', 'iu': 'yêu', 'vs': 'với', 'ms': 'mới',
        'zề': 'về', 'rùi': 'rồi', 'nỳ': 'này', 'tru': 'thật', 'h m': 'hỏi mà',
        'lm': 'làm', 'phờ': 'phê', 'á đù': 'bất ngờ', 'đhs': 'đéo hiểu sao', 'hihi': 'cười nhỏ',
        'tíh': 'tính', 'èo': 'khó chịu', 'trời': 'chúa', 'gato': 'ghen ăn tức ở', 'vlol': 'vãi cười',
        'ăn lol': 'ăn không ngon', 'cứt': 'rất tệ', 'chịch': 'quan hệ tình dục', 'địt': 'quan hệ tình dục',
        'dm': 'đồ ngu', 'vkl': 'vãi cực luôn', 'vcc': 'vãi cả cười', 'cmm': 'câm mồm', 'dcm': 'đồ ngu ngốc',
        'cc': 'vãi thật', 'súc vật': 'người xấu', 'óc chó': 'người ngốc nghếch', 'đầu bò': 'người ngu',
        'lợn': 'người ngốc', 'ngu lol': 'rất ngu', 'hâm': 'người không bình thường', 'điên': 'người không tỉnh táo',
        'bẩn': 'rất dơ', 'thối': 'rất hôi', 'lầy': 'rất hài', 'nhây': 'rất dai dẳng', 'kaka': 'haha',
        'ahihi': 'cười lớn', 'ahuhu': 'rất buồn', 'kute': 'dễ thương', 'xinh vl': 'rất xinh', 'đẹp trai vl': 'rất đẹp trai',
        'vl cmm': 'rất lắm', 'ngu vcc': 'rất ngu', 'gato vcc': 'ghen ăn tức ở vãi', 'cmm vl': 'rất nhiều',
        'ahihi đồ ngốc': 'cười vào người ngốc', 'hụt': 'thiếu hụt', 'rớt': 'thất bại', 'lỗi': 'sai sót',
        'kool': 'rất chất', 'phét': 'không thật', 'nói phét': 'nói dối', 'bốc phét': 'bịa chuyện', 'chém gió': 'nói phóng đại',
        'bựa': 'rất kỳ cục', 'đểu': 'rất xấu', 'đồ đểu': 'người xấu', 'khốn': 'rất tệ', 'vô đạo đức': 'rất xấu',
        'cực phẩm': 'rất tốt', 'bá đạo': 'rất giỏi', 'ngon lành': 'rất tốt', 'kém chất lượng': 'rất tệ', 'toang': 'thất bại',
        'xàm': 'không có ý nghĩa', 'xàm lol': 'rất nhảm nhí', 'nhảm': 'vô nghĩa', 'dỗi': 'giận dỗi', 'giận hờn': 'không vui',
        'hận': 'rất ghét', 'phũ': 'rất lạnh lùng', 'đào hoa': 'rất hấp dẫn', 'chảnh': 'kiêu căng', 'kiêu': 'rất tự cao',
        'sến': 'rất lãng mạn', 'ngôn tình': 'rất tình cảm', 'gớm': 'rất xấu', 'tởm': 'rất ghê', 'rùng mình': 'rất sợ',
        'tởm lol': 'rất kinh khủng', 'kinh tởm': 'rất ghê', 'dị': 'rất lạ', 'dị hợm': 'rất kỳ quái', 'quái': 'rất lạ',
        'quái đản': 'rất kỳ dị', 'quái dị': 'rất xấu', 'chán đời': 'rất buồn', 'tự tử': 'kết thúc cuộc sống',
        'mất tích': 'không thấy đâu', 'đi bụi': 'bỏ nhà đi', 'cờ bạc': 'đánh bạc', 'nghiện ngập': 'rất tệ', 'đập đá': 'sử dụng ma túy',
        'chơi bời': 'rất phóng túng', 'ăn chơi': 'rất hưởng thụ', 'bạo lực': 'rất mạnh', 'phang': 'đánh mạnh', 'vả': 'tát',
        'đấm': 'đánh', 'đá': 'chân đánh', 'chém': 'sử dụng dao', 'hành hung': 'đánh đập', 'ẩu đả': 'đánh nhau',
        'đánh lộn': 'đánh nhau', 'hỗn chiến': 'đánh nhau đông người', 'bạo loạn': 'rất hỗn loạn'
    }

In [4]:
import re
from pyspark.sql.functions import col, lower, udf
from pyspark.sql.types import StringType

def remove_emoji(text):
    if text is None:
        return text
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F1E0-\U0001F1FF"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

remove_emoji_udf = udf(remove_emoji, StringType())


def replace_teencode(text):
    words = text.split()
    replaced = [teencode_dict.get(w.lower(), w) for w in words]
    return " ".join(replaced)

replace_teencode_udf = udf(replace_teencode, StringType())

def preprocess_to_single_file(input_path, output_dir, final_name="final.csv"):
    """
    1) Đọc CSV từ input_path
    2) Bỏ null và dòng cột text, label == 'NaN'
    3) lowercase text
    4) remove emoji
    5) replace teencode
    6) gộp thành 1 file CSV duy nhất => rename part-xxx => final_name
    """
    import os, glob, shutil

    df = spark.read.csv(input_path, header=True, inferSchema=True)

    df = df.dropna(subset=["text", "label"])

    df = df.filter((col("text") != "NaN") & (col("label") != "NaN"))

    df = df.withColumn("text", lower(col("text")))

    df = df.withColumn("text", remove_emoji_udf(col("text")))

    df = df.withColumn("text", replace_teencode_udf(col("text")))

    df.coalesce(1).write.mode("overwrite").option("header", True).csv(output_dir)

    part_files = glob.glob(output_dir + "/part-*.csv")
    if part_files:
        part_file = part_files[0]
        new_path = os.path.join(output_dir, final_name)
        shutil.move(part_file, new_path)

    success_file = os.path.join(output_dir, "_SUCCESS")
    if os.path.exists(success_file):
        os.remove(success_file)

    print(f"[DONE] Đã xử lý xong. File cuối cùng: {output_dir}/{final_name}")


In [5]:
train_input_path = "/content/drive/MyDrive/data/train.csv"
test_input_path  = "/content/drive/MyDrive/data/test.csv"

train_output_dir = "/content/drive/MyDrive/data/train_preprocessed"
test_output_dir  = "/content/drive/MyDrive/data/test_preprocessed"



preprocess_to_single_file(train_input_path, train_output_dir, final_name="train_final.csv")
preprocess_to_single_file(test_input_path,  test_output_dir,  final_name="test_final.csv")


Py4JJavaError: An error occurred while calling o82.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (LazyFeel executor driver): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to file:/d:/Project/Data.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:775)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:420)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:100)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:893)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:893)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367

In [ ]:
processed_train = spark.read.csv("/content/drive/MyDrive/data/train_preprocessed/train_final.csv",
                                 header=True, inferSchema=True)

print("Schema sau khi xử lý:")
processed_train.printSchema()

print("Xem 10 dòng đầu:")
processed_train.show(10, truncate=False)


Schema sau khi xử lý:
root
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)

Xem 10 dòng đầu:
+------------------------------------------------------------------------------------------------------------+-----+
|text                                                                                                        |label|
+------------------------------------------------------------------------------------------------------------+-----+
|em được làm fan cứng luôn rồi nè ❤️ reaction quá hay quá cute coi mấy giờ này quá hợp lí =]]]               |0    |
|đúng là bọn mắt híp lò xo thụt :))) bên việt nam t cái này ra cách đây 10 năm rồi và bọn t gọi là cái l :)))|2    |
|đậu văn cường giờ giống thằng sida hơn à                                                                    |0    |
|côn đồ cục súc vô nhân tính đề nghi vn. nhà nước vn ban thưởng.                                             |2    |
|từ lý thuyết đến thực hành là cả 1 câu chuyện dài =))         